In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os

def fetch_dvol_data_paginated(start_date, end_date, resolution):
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    end_time = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp() * 1000)
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    
    # Convert resolution to integer if passed as string
    resolution = int(resolution)
    interval = resolution * 1000 * 1000  # Backward time window (e.g., 1000 rows)

    all_data = []
    
    while end_time > start_time:
        # Calculate the start time window for the next request
        start_window = end_time - interval
        if start_window < start_time:
            start_window = start_time
        
        params = {
            'currency': 'BTC',
            'start_timestamp': start_window,
            'end_timestamp': end_time,
            'resolution': resolution
        }
        
        try:
            response = requests.get(url, params=params)
            data = response.json()
            
            # Debugging: Print response to check for issues
            print("API Response:", data)

            if 'result' in data and 'data' in data['result']:
                raw_data = data['result']['data']
                
                if not raw_data:
                    print("No more data available.")
                    break
                
                # Convert to DataFrame
                df = pd.DataFrame(raw_data, columns=['timestamp', 'open', 'high', 'low', 'close'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                all_data.append(df)
                
                # Update end time to fetch older data
                end_time = int(df['timestamp'].min().timestamp() * 1000) - resolution * 1000
            else:
                print("Unexpected API response structure or no data returned.")
                break

        except Exception as e:
            print(f"Error fetching data: {e}")
            break

    if all_data:
        final_df = pd.concat(all_data).drop_duplicates(subset='timestamp').sort_values(by='timestamp')
        return final_df
    else:
        return None


def save_or_update_csv(df, csv_filename):
    if os.path.exists(csv_filename):
        existing_df = pd.read_csv(csv_filename)
        
        # Ensure timestamp is converted to datetime in both DataFrames
        existing_df['timestamp'] = pd.to_datetime(existing_df['timestamp'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Combine and deduplicate
        combined_df = pd.concat([existing_df, df]).drop_duplicates(subset='timestamp').sort_values(by='timestamp')
        combined_df.to_csv(csv_filename, index=False)
        print(f"Updated {csv_filename}")
    else:
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")


In [2]:
if __name__ == "__main__":
    start_date = "2024-06-01"
    end_date = "2025-03-09"
    resolution = "3600"  # 1hr resolution 3600 sec
    
    # Fetch paginated DVOL data
    dvol_data = fetch_dvol_data_paginated(start_date, end_date, resolution)
    
    if dvol_data is not None:
        csv_path = r"C:\Users\USER\Documents\Tradingstuff"
        csv_filename = os.path.join(csv_path if csv_path else '.', "BTC_IV_Deribit_hour.csv")
        save_or_update_csv(dvol_data, csv_filename)



API Response: {'jsonrpc': '2.0', 'result': {'data': [[1737853200000, 56.98, 56.98, 56.82, 56.83], [1737856800000, 56.83, 56.83, 56.65, 56.69], [1737860400000, 56.69, 56.8, 56.63, 56.78], [1737864000000, 56.78, 57.0, 56.75, 56.9], [1737867600000, 56.9, 57.23, 56.9, 57.23], [1737871200000, 57.23, 57.25, 57.18, 57.23], [1737874800000, 57.23, 57.24, 57.19, 57.21], [1737878400000, 57.21, 57.27, 57.05, 57.12], [1737882000000, 57.12, 57.4, 56.92, 57.39], [1737885600000, 57.39, 57.56, 57.03, 57.56], [1737889200000, 57.56, 57.61, 57.4, 57.56], [1737892800000, 57.56, 57.58, 57.5, 57.53], [1737896400000, 57.53, 57.61, 57.31, 57.59], [1737900000000, 57.59, 57.59, 57.51, 57.53], [1737903600000, 57.53, 57.53, 57.0, 57.13], [1737907200000, 57.13, 57.3, 56.81, 56.83], [1737910800000, 56.83, 57.37, 56.82, 57.28], [1737914400000, 57.28, 57.28, 57.02, 57.07], [1737918000000, 57.07, 57.21, 56.67, 57.16], [1737921600000, 57.16, 57.21, 56.78, 56.85], [1737925200000, 56.85, 57.28, 56.65, 56.65], [17379288000